# Dimensionality Recution
Typically, when we train we have instances that have trillions upon billions of features per instance. Not only does it make training really slow, it could also make it harder to find a good solution. This is often refered as the _curse of dimensionality_.

Fortunatly for us, it's often possible to reduce the dimensions!!! For example, the edges of the MNIST set could be trimmed since they hold no useful information. If two neighboring pixels are highly correlated, they can be merged into a single pixel. Keep in mind that reducing features leads to information loss! This may lead to slightly worse performance. That aside, dimensionality reduction can lead to data visualization(AKA _DataViz_). Here we will look at three techniques: PCA, Kernel PCA, and LLE.

## The Curse of Dimensionality
High dimensional stuff be cray cray fam. Look at the book for diz. Keep in mind, the greater the dimensions, the greater the risk of overfitting it! One theoretical solution would be to increase the training set to a rediculous size... however this in practice is not possible.

## Main Approaches for Dimensionality Reduction
Before we look at specific reduction algos, let's look at two main approaches: projection and Manifold Learning

### Projection
In most real-world problems, training instances are _not_ spread out uniformly across all dimensions. Many features are almost constant, while others are highly correlated, hence they actually lie within a lower-dimensional _subspace_. Projection is not the best approach however. Take a look at the famous _Swiss roll_ toy dataset. Dropping a plane is bad, you need to unroll it!

### Manifold Learning
Many dimensionality reduction algos work by modeling the _manifold_ on which the training instances lie upon. This is called _Manifold Learning_. This relies onthe _manifold assumtion_, also called _manifold hypothesis_, which holds that most real-world highdimensional data lie closer to a much lower-dimensional manifold.

Again, think about the MNIST set: all handwriten digets have some similarities. They have connected lines, white borders, and are more or less centered. If we had instead randomly generated images, we probably wouldn't be able to reduce the dimensions, given that constraints tend to lead to a dataset being projected onto a lower dimension manifold.

The manifold assumption also has another assumption being that the task at hand will be simpler if expressed in a lower dimension. In short, it all depends on how the data set is!

## PCA
_Principal Component Analysis_(PCA) is the most popular dimensionality reduction algo. It identifies the hyperplane that lies closest to the data, then projects the data onto it.

### Preserving the Variance
You need to choose the right plane and axis to maintain the variance! This is to minizmize the mean squared distance between the original and it's projection onto the axis. This is the idea behind PCA.

### Principal Components
PCA identifies the axis that accounts for the largest amount of variance in the set. It then recursivly finds axis orthoganl to the previous axises. The unit vector that describes the ith axis is called the ith _principal component_. So how do you find the PCs of the set? You do so using a technique called _Singular Value Decomposition_(SVD). That decomposes the trainng set X to the three matricies U E and V^T which contains all of the principal components. Like the following...

In [ ]:
# You need to generate a dataset for X first!

X_centered = X - X.mean(axis=0)
U, s, V = np.linalg.svd(X_centered)
c1 = V.T[:, 0]
c2 = V.T[:, 1]

### Projecting Down to d Dimensions
Once all principal components have be identified, you can reduce the dimensionality of the dataset down to _d_ dimensions by projecting it onto the first _d_ principal components. To project the set onto the hyperplane, simply do the dot maxtrix of the set and the matrix W_d_, defined by the first _d_ principal components. The following code does just that!

In [ ]:
W2 = V.T[:, :2]
X2D = X_centered.dot(W2)

Of course, there's the Scikit version of doing things...

In [ ]:
from sklearn.decompositions import PCA

pca = PCA(n_components = 2)
X2D = pca.fit_transform(X)

### Explained Variance Ratio
Another useful piece of information, the _explained variance ratio_ of each principal component. It indicates the portion of the dataset's variance that lies along the axis of each principal component. Here's a sample...

In [ ]:
print(pca.explained_variance_ratio_)

### Choosing the Right Number of Dimensions
Instead of arbitraraly choosing the number of dimensions to reduce down to, it's best to choose a number that adds up to a sufficently large portion of variance (95%)... unless it's for data visualization.

The following finds a _d_ that maintaince a certain amount of variance.

In [ ]:
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1

Of course, there's a more direct way to get the ratio you desire...

In [ ]:
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X)

Another way to find the optimal number of dimesions is to plot the darn thing and find the dimention size that you want!

### PCA for Compression
By applying PCA to the dataset, you can achieve resonable compression. For example the MNIST dataset could be compressed by 20%! It's possible to recover the reduced dataset, but there will be some information loss. THe following is an example of PCA to 154 components then recovering the original data.

In [ ]:
pca = PCA(n_components=154)
X_mnist_reduced = pca.fit_transform(X_mnist)
X_mnist_recovered = pca.inverse_transform(X_mnist_reduced)

### Incremental PCA
One problem of PCA is that it requires the whole set to fit in memory! Luckly, there _Incremental PCA_(IPCA) is a thing to split the set into mini-batches and feed it to the algo one batch at a time. This can also be used for online algos!

Here's an example using the MNIST dataset!

In [ ]:
from sklearn.decomposition import IncrementalPCA

n_batches = 100
inc_pca = InvcrementalPCA(n_components=154)
for X_batch in np.array_split(X_mnist, n_batches):
    inc_pca.partial_fit(X_batch)

X_mnist_reduced = inc_pca.transform(X_mnist)

Of course, you can instead read in data from memory only when it needs it such as the following...

In [ ]:
X_mm = np.memmap(filename, dtype="float32", mode="readonly", shape=(m, n))

batch_size = m // n_batches
inc_pca = IncrementalPCA(n_components=154, batch_size=batch_size)
inc_pca.fit(X_mm)

### Randomized PCA
This is a stochastic algo that finds an approximation of the first _d_ principal components. It's complexity is:
O(m * d^2) + O(d^3) instead of the same, swapping _d_ for _n_. Hence, when the improvment in speed is massive when _d_ is much smaller than _n_.

In [ ]:
rnd_pca = PCA(n_components=154, svd_solver="randomized")
X_reduced rnd_pca.fit_transform(X_mnist)

### Kernel PCA
We can apply the Kernel trick discussed in chapter 5 to perform complex nonlinear projections for dimensionality reduction. It's good at perversing clusters of instances and sometimes even unrolling datasets that lie close to a twisted manifold.

In [ ]:
from sklearn.decomposition import KernelPCA

rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

### Selecting a Kernel and Tuning Hyperparameters
Since kPCA is an unsupervised algo, there's no obvious performance measure for it. However, since this is often a preparation step for a supervised learning task you can use grid search to find the best kernel and hyperparameters. Here's a example:

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

clf = Pipeline([
    ("kpca", KernelPCA(n_components=2)),
    ("log_reg", LogisticRegression())
])

param_grid = [{
    "kpca__gamma": np.linspace(0.03, 0.05, 10),
    "kpca__kernel": ["rbf", "sigmoid"]
}]

grid_search = GridSearchCV(clf, param_grid, cv=3)
gird_search.fit(X, y)

Another approach, this time entierly unsupervised, is to choose the kernel with the least reconstruction error. This however is not as easy as linear PCA... take a look at the swiss roll image on 220. Since the feature map essentially has infinite dimensions, it can only map an approximation. Big oof.

As for how to perform reconstruction, one solution is to train a supervised regression model, with  projected instances as the training set and the original instances as the targets. Scikit can do this automagically!

In [ ]:
rbf_pca = KernelPCA(n_components=2, kernel="rbf", gamma=0.0433, fit_inverse_transform=True)
X_reduced = rbf_pca.fit_transform(X)
X_preimage = rbf_pca.inverse_transform(X_reduced)

And then compute the error...

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(X, X_preimage)

## LLE
_Locally Linear Embedding_(LLE) is another _nonlinear dimensionality reduction_(NLDR) technique. It's a Manifold Learning technique that doesn't rely on projections like the previous ones!!! It works by first measuring how each training instance linearly relates to its closest neighbors and then lookikng for a low-dimensional representation of the set in which the local relations are best preserved. This makes it particularly good at unrolling these swiss rolls.

In the following example we use Scikit's `LocallyLinearEmbedding` class to unroll the swiss roll! Note that distances are _not_ preserved on a larger scale: the left part of the roll is squeezed while the right part is stretched!

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)

Refer to the book for the implementation and the computational complexity, just know that this doesn't scale well with very large datasets!!!

## Other Dimensionality Reduction Techniques
- _Multidimensional Scaling_(MDS) reduces dimensionality while trying to preserve the distances between instances
- _Isomap_ Creates a map by connecting it's nearest neighbors, then reduces dimensionality while trying to preserve the _geometric distances_ between instances.
- _t-Distributed Stochastic Neighbor Embedding_ (t-SNE) reduces dimensionality while trying to keep similar instances close and dissimilar instances apart. Mostly used for visualization, in partucular to visualize clusters of instances in high dimensional space.
- _Linear Discriminant Analysis_(LDA) is actually a classification algo, but during training, it learns the most discriminative axes between the classes, and these axes can be used to define a hyperplane onto which to project the data. The benefit is that this projection will keep the classes as far as possible, so LDA is a good technique to reduce dimensionality before running another classification algo such as a SVM classifier.